# 分析
切り出したショットデータを取得し、分析します。  
以下は特定ショットのデータを取得/確認し、特徴点（荷重開始点/最大荷重点/破断点）を抽出、その後全ショットから特徴点を抽出するサンプルコードです。抽出した特徴点はElasticsearchに格納されます。

In [2]:
# ライブラリインポート
import os, sys
sys.path.append(os.path.join(os.getcwd(), '../src/'))
from data_reader.data_reader import DataReader
from elastic_manager.elastic_manager import ElasticManager
sys.path.append(os.path.join(os.getcwd(), '../src/analyze/'))
from analyze.h_one_extract_features import *
from analyze.analyze import *

Loading BokehJS ...

In [3]:
# ログ設定
LOG_FILE = os.path.join(
    common.get_config_value(common.APP_CONFIG_PATH, "log_dir"), "analyze/analyze.log"
)

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    handlers=[
        logging.handlers.RotatingFileHandler(
            LOG_FILE, maxBytes=common.MAX_LOG_SIZE, backupCount=common.BACKUP_COUNT
        ),
        logging.StreamHandler(),
    ],
)

In [4]:
# ショットデータインデックス一覧
ElasticManager.show_indices(index="shots-*-data")

,index,docs.count,store.size
0,shots-20201201010000-data,0,0
1,shots-20210201151814-data,0,0
2,shots-fixed-data,0,0


In [5]:
# 対象データ設定
target = "20201201010000"
shots_data_index = "shots-" + target + "-data"
shots_meta_index = "shots-" + target + "-meta"

# 1ショットで特徴点抽出

## ショット1番を取得して確認

In [6]:
dr = DataReader()

shot_number = 1
shot_df = dr.read_shot(shots_data_index, shot_number=shot_number)
shot_df.head()

""


In [15]:
# ショットメタデータを確認
shots_meta_df = dr.read_shots_meta(shots_meta_index)
shots_meta_df.head()

,shot_number,spm,num_of_samples_in_cut_out
0,1,80.598572,3125.0
1,2,80.613753,3113.0
2,3,80.594235,3129.0
3,4,80.636509,3097.0
4,5,80.582338,3140.0


In [16]:
# ショットの数を確認
print(f"ショット数：{len(shots_meta_df)}")

ショット数：2980


In [18]:
# spm
spm = shots_meta_df[shots_meta_df.shot_number == 1].spm
spm

## 特徴点抽出
1ショットから最大荷重点、荷重開始点、破断点を取得します。  

***shot_data***: 対象ショットデータ  
***spm***: spm  
***func***：適用するロジック  
***sub_func***：サブロジック  
***disp_chart***: グラフ表示

In [1]:
# 最初のショットの最大荷重点、荷重開始点、破断点を確認
max_load_indicies, max_load_values, debug_values = extract_features(shot_data=shot_df, spm=spm, func=max_load, shot=shot_number, disp_chart=True)
load_start_indicies, load_start_values, debug_values = extract_features(shot_df, spm, load_start2, shot=shot_number, disp_chart=True)
break_load_indicies, break_load_values, debug_values = extract_features(shot_df, spm, breaking_varmax29, sub_func=narrowing_var_ch,shot=shot_number, disp_chart=True)

NameError: name 'extract_features' is not defined

In [7]:
# 各特徴点のインデックス番号と値を確認
print(f"【最大荷重点】インデックス番号：{max_load_indicies}, 値：{max_load_values}")
print(f"【荷重開始点】インデックス番号：{load_start_indicies}, 値：{load_start_values}")
print(f"【破断点】　　インデックス番号：{break_load_indicies}, 値：{break_load_values}")

NameError: name 'max_load_indicies' is not defined

# 全ショットにロジック適用

## データ読み込み
指定したインデックスの全ショットデータを読み込みます。これには時間がかかります。  
時間目安：10,000,000件読み込みで約3分

In [10]:
%%time
# 全ショットデータ読み込み
target = "20201201010000"
shots_data_index = "shots-" + target + "-data"

dr = DataReader()
shots_df = dr.multi_process_read_all(shots_data_index)

2021-02-12 11:46:40,195 [INFO] データを全件取得します。データ件数が多い場合、長時間かかる場合があります。
2021-02-12 11:46:40,198 [INFO] データ件数: 9655193
2021-02-12 11:46:40,199 [INFO] Data read start. data_count: 9655193.


CPU times: user 1min 7s, sys: 4.15 s, total: 1min 11s
Wall time: 3min 17s


In [11]:
# ショットメタデータ読み込み
shots_meta_index = "shots-" + target + "-meta"
shots_meta_df = dr.read_shots_meta(shots_meta_index)

## 全ショットから特徴点抽出
全ショットから最大荷重点、荷重開始点、破断点を抽出し、Elasticsearchに保存します。  

***target***: 対象ショットデータ（yyyyMMddHHMMSS）  
***shots_df***: ショットデータ  
***shots_meta_df***: ショットメタデータ  
***feature***: 抽出する特徴点名（max/start/break）  
***func***：適用するロジック  
***sub_func***：サブロジック  

保存先インデックスは以下になります。  
* 最大荷重点：shots-yyyyMMddHHMMSS-max-point
* 荷重開始点：shots-yyyyMMddHHMMSS-start-point
* 破断点：shots-yyyyMMddHHMMSS-break-point

In [13]:
# 最大荷重点
apply(
    target=target,
    shots_df=shots_df,
    shots_meta_df=shots_meta_df,
    feature="max",
    func=max_load,
    sub_func=None,
)

2021-02-12 11:01:00,320 [INFO] apply start.
2021-02-12 11:01:00,408 [INFO] delete index 'shots-20201201010000-max-point' finished. result: {'acknowledged': True}
2021-02-12 11:01:18,421 [INFO] apply finished.


In [14]:
# 荷重開始点
apply(
    target="20201201010000",
    shots_df=shots_df,
    shots_meta_df=shots_meta_df,
    feature="start",
    func=load_start2,
    sub_func=None,
)

2021-02-12 11:01:18,429 [INFO] apply start.
2021-02-12 11:01:18,526 [INFO] delete index 'shots-20201201010000-start-point' finished. result: {'acknowledged': True}
2021-02-12 11:01:45,505 [INFO] apply finished.


In [12]:
# 破断点
apply(
    target="20201201010000",
    shots_df=shots_df,
    shots_meta_df=shots_meta_df,
    feature="break",
    func=breaking_var_vrms,
    sub_func=ef.narrowing_var_ch,
)

2021-02-12 12:56:44,650 [INFO] apply start.
2021-02-12 12:59:08,769 [INFO] apply finished.
